In [1]:
from deap import tools, base, creator, algorithms
import jax
from diversity_algorithms.controllers.fixed_structure_nn_flax import SimpleNeuralControllerFlax
import jax.numpy as jnp
from functools import partial
from diversity_algorithms.environments.brax_env import EvaluationFunctor
from diversity_algorithms.algorithms.novelty_search import set_creator
from diversity_algorithms.environments.behavior_descriptors import feet_contact_descriptor
creator.create("FitnessMax", base.Fitness, weights=(1.0,)*1)
import numpy as np
creator.create("Individual", np.ndarray, typecode="d", fitness=creator.FitnessMax)
set_creator(creator)


from scipy.spatial import KDTree
import numpy as np

import pickle
import time
from diversity_algorithms.algorithms.utils import *
from diversity_algorithms.analysis.population_analysis import *
from diversity_algorithms.analysis.data_utils import *

from diversity_algorithms.algorithms.novelty_management import *

import alphashape
from shapely.geometry import Point, Polygon, LineString
from diversity_algorithms.algorithms.jax_utils import *
from diversity_algorithms.environments.brax_env import create

from IPython.display import HTML
from brax.v1.io import html


%load_ext autoreload
%autoreload 2


In [2]:
env = create("ant-uni", episode_length=300)
random_key = jax.random.PRNGKey(0)

In [3]:
controller = SimpleNeuralControllerFlax(env.observation_size, env.action_size, n_hidden_layers=2, n_neurons_per_hidden=64)
eval_functor = EvaluationFunctor("ant", output="reward_survive", controller=controller, bd_function=feet_contact_descriptor)

Environment set to ant


In [4]:
def dist_to_shape(pp, s):
	p=Point(pp)
	d=p.distance(s)
	if (d==0.0):
		d=-p.distance(s.exterior)
	return d

def dist_to_shapes(pp, ls):
	if (not hasattr(ls, '__iter__')):
		ls=[ls] 
	p=Point(pp)
	imin=-1
	dmin=sys.float_info.max
	for i in range(len(ls)):
		d=p.distance(ls[i])
		if (d<dmin):
			imin=i
			dmin=d
	if (dmin==0.0):
		d=-p.distance(ls[i].exterior)
	else:
		d=dmin
	return d

In [5]:
params = {"nb_gen": 50,
		  "pop_size": 1000,
			"geno_type":"realarray", 
		  "variant":"NS", 
		  "ind_size":controller.n_weights, 
		  "eta_m":15.0,
		  "indpb":0.1,
		  "mutpb":0,
		  "cxpb":1,
		  "min":-1,
		  "max":1,
		  "k":15,
		  "add_strategy":"novel",
		  "lambda_nov":60,
		  "verbosity":"none",
	}

In [6]:
toolbox = base.Toolbox()
toolbox.register("population", init_pop_controller, controller=controller)
toolbox.register("mutate", mutate, eta=params["eta_m"], min_val=params["min"], max_val=params["max"], indpb=params["indpb"])
toolbox.register("mate", cxBLend, alpha=params["indpb"])
toolbox.register("map_eval", eval_functor)
v=str(params["variant"])
variant=v.replace(",","")
if (variant == "NS"): 
	toolbox.register("select", selBest, fit_attr='novelty')
elif (variant == "Fit"):
	toolbox.register("select", selBest, fit_attr='fitness')
elif (variant == "Random"):
	toolbox.register("select", random.sample)
elif (variant == "DistExplArea"):
	toolbox.register("select", selBest, fit_attr='dist_to_explored_area')
else:
	print("Variant not among the authorized variants (NS, Fit, Random, DistExplArea), assuming multi-objective variant")
	toolbox.register("select", tools.selNSGA2)

In [7]:
population, random_key = toolbox.population(random_key, params["pop_size"])

In [8]:
fit, bd, random_key = toolbox.map_eval(jnp.asarray(population), random_key)

In [9]:
bd_dimension = bd.shape[-1]

for ind, f, b in zip(population, fit, bd):
	ind.fit = f # fit is an attribute just used to store the fitness value
	ind.parent_bd = None
	ind.bd = b
	ind.id = generate_uuid()
	ind.parent_id = None

for ind in population:
	ind.am_parent=0

In [10]:
archive=updateNovelty(population,population,None,params)
alpha_shape = None
if bd_dimension == 2 or bd_dimension == 3:
	alpha_shape = alphashape.alphashape(archive.all_bd, 5)
isortednov=sorted(range(len(population)), key=lambda k: population[k].novelty, reverse=True)
varian=params["variant"].replace(",","")

In [11]:
if ("+" in variant):
	emo=True
else:
	emo=False

In [12]:
for i,ind in enumerate(population):
	if alpha_shape:
		ind.dist_to_explored_area = dist_to_shapes(ind.bd,alpha_shape)
	ind.rank_novelty=isortednov.index(i)
	ind.dist_to_parent=0
	if (emo): 
		if (varian == "NS+Fit"):
			ind.fitness.values = (ind.novelty, ind.fit)
		elif (varian == "NS+BDDistP"):
			ind.fitness.values = (ind.novelty, 0)
		elif (varian == "NS+Fit+BDDistP"):
			ind.fitness.values = (ind.novelty, ind.fit, 0)
		else:
			print("WARNING: unknown variant: "+variant)
			ind.fitness.values=ind.fit
	else:
		ind.fitness.values=ind.fit
	# if it is not a multi-objective experiment, the select tool from DEAP 
	# has been configured above to take the right attribute into account
	# and the fitness.values is thus ignored
gen=0	

In [13]:
lambda_ = 2000

In [14]:
offspring, random_key = varOr(random_key, population, toolbox, lambda_, params["cxpb"], params["mutpb"])

In [15]:
fit, bd, random_key = toolbox.map_eval(jnp.asarray(offspring), random_key)

In [16]:
for ind, f, b in zip(offspring, fit, bd):
	ind.fit = f
	ind.fitness.values = f
	ind.parent_bd = ind.bd
	ind.parent_id = ind.id
	ind.id = generate_uuid()
	ind.bd = b
for ind in population:
	ind.am_parent=1
for ind in offspring:
	ind.am_parent=0
	
pq=population+offspring


pop_for_novelty_estimation=pq

In [17]:
archive = updateNovelty(pq,offspring,archive,params, pop_for_novelty_estimation)
if bd_dimension == 2 or bd_dimension == 3:
	alpha_shape = alphashape.alphashape(archive.all_bd, 5)

In [18]:
nov = [ind.novelty for ind in pq]
isortednov = np.argsort(nov)[::-1]
rank = np.empty_like(isortednov)
rank[isortednov] = np.arange(len(isortednov))

In [19]:
for i,ind in enumerate(pq):
	if bd_dimension == 2 or bd_dimension == 3:
		ind.dist_to_explored_area=dist_to_shapes(ind.bd,alpha_shape)
	ind.rank_novelty = rank[i]
	#print("Indiv #%d: novelty=%f rank=%d"%(i, ind.novelty, ind.rank_novelty))
	if (ind.parent_bd is None):
		ind.dist_to_parent=0
	else:
		ind.dist_to_parent=np.linalg.norm(np.asarray(ind.bd)-np.asarray(ind.parent_bd))
	if (emo):
		if (varian == "NS+Fit"):
			ind.fitness.values = (ind.novelty, ind.fit)
		elif (varian == "NS+BDDistP"):
			if (ind.parent_bd is None):
				bddistp=0
			else:
				bddistp=np.linalg.norm(np.asarray(ind.bd) - np.asarray(ind.parent_bd))
			ind.fitness.values = (ind.novelty, bddistp)
		elif (varian == "NS+Fit+BDDistP"):
			if (ind.parent_bd is None):
				bddistp=0
			else:
				bddistp=np.linalg.norm(np.asarray(ind.bd) - np.asarray(ind.parent_bd))
			ind.fitness.values = (ind.novelty, ind.fit, bddistp)
		else:
			print("WARNING: unknown variant: "+variant)
			ind.fitness.values=ind.fit
	else:
		ind.fitness.values=ind.fit

In [20]:
if ("," in variant):
	population[:] = toolbox.select(offspring, params["pop_size"])		
else:
	population[:] = toolbox.select(pq, params["pop_size"])  

In [21]:
offspring, random_key = varOr(random_key, population, toolbox, lambda_, params["cxpb"], params["mutpb"])

In [22]:
best_ind = []

In [23]:
for gen in range(1, params["nb_gen"] + 1):
	print("Generation %d" % gen)
	t = time.time()
	offspring, random_key = varOr(random_key, population, toolbox, lambda_, params["cxpb"], params["mutpb"])
	print("varOr time: ", time.time() - t)	
	# Evaluate the individuals with an invalid fitness
	t=time.time()
	fit, bd, random_key = toolbox.map_eval(jnp.asarray(offspring), random_key)
	print("map_eval time: ", time.time() - t)

	for ind, f, b in zip(offspring, fit, bd):
		ind.fit = f
		ind.fitness.values = f
		ind.parent_bd = ind.bd
		ind.parent_id = ind.id
		ind.id = generate_uuid()
		ind.bd = b

	for ind in population:
		ind.am_parent=1

	for ind in offspring:
		ind.am_parent=0
		
	pq = population+offspring
	pop_for_novelty_estimation = pq
	t=time.time()
	archive = updateNovelty(pq, offspring, archive, params, pop_for_novelty_estimation)
	print("updateNovelty time: ", time.time() - t)
	if bd_dimension == 2 or bd_dimension == 3:
		alpha_shape = alphashape.alphashape(archive.all_bd, 5)
	
	# Compute the novelty rank
	nov = [ind.novelty for ind in pq]
	isortednov = np.argsort(nov)[::-1]
	rank = np.empty_like(isortednov)
	rank[isortednov] = np.arange(len(isortednov))
	
	for i,ind in enumerate(pq):
		if alpha_shape:
			ind.dist_to_explored_area = dist_to_shapes(ind.bd,alpha_shape)
		ind.rank_novelty = rank[i]
		#print("Indiv #%d: novelty=%f rank=%d"%(i, ind.novelty, ind.rank_novelty))
		if (ind.parent_bd is None):
			ind.dist_to_parent=0
		else:
			ind.dist_to_parent=np.linalg.norm(np.asarray(ind.bd)-np.asarray(ind.parent_bd))
		if (emo):
			if (varian == "NS+Fit"):
				ind.fitness.values = (ind.novelty, ind.fit)
			elif (varian == "NS+BDDistP"):
				if (ind.parent_bd is None):
					bddistp=0
				else:
					bddistp=np.linalg.norm(np.asarray(ind.bd) - np.asarray(ind.parent_bd))
				ind.fitness.values = (ind.novelty, bddistp)
			elif (varian == "NS+Fit+BDDistP"):
				if (ind.parent_bd is None):
					bddistp=0
				else:
					bddistp=np.linalg.norm(np.asarray(ind.bd) - np.asarray(ind.parent_bd))
				ind.fitness.values = (ind.novelty, ind.fit, bddistp)
			else:
				print("WARNING: unknown variant: "+variant)
				ind.fitness.values=ind.fit
		else:
			ind.fitness.values=ind.fit
	if (verbosity(params)):
		print("Gen %d"%(gen))
	else:
		if(gen%100==0):
			print(" %d "%(gen), end='', flush=True)
		elif(gen%10==0):
			print("+", end='', flush=True)
		else:
			print(".", end='', flush=True)
	
	best_ind.append(selBest(pq, 1, fit_attr="fitness")[0])
	# Select the next generation population
	if ("," in variant):
		population[:] = toolbox.select(offspring, params["pop_size"])		
	else:
		population[:] = toolbox.select(pq, params["pop_size"])		


Generation 1
varOr time:  0.2702910900115967
map_eval time:  0.5309240818023682
updateNovelty time:  0.4350252151489258
.Generation 2
varOr time:  0.24020910263061523
map_eval time:  0.5263488292694092
updateNovelty time:  0.4229700565338135
.Generation 3
varOr time:  0.31922078132629395
map_eval time:  0.5405852794647217
updateNovelty time:  0.43637752532958984
.Generation 4
varOr time:  0.2762625217437744
map_eval time:  0.528538703918457
updateNovelty time:  0.41707324981689453
.Generation 5
varOr time:  0.40997934341430664
map_eval time:  0.5314359664916992
updateNovelty time:  0.42789483070373535
.Generation 6
varOr time:  0.2695276737213135
map_eval time:  0.52907395362854
updateNovelty time:  0.42067599296569824
.Generation 7
varOr time:  0.2522571086883545
map_eval time:  0.5360219478607178
updateNovelty time:  0.42688632011413574
.Generation 8
varOr time:  0.2521061897277832
map_eval time:  0.52372145652771
updateNovelty time:  0.4127962589263916
.Generation 9
varOr time:  0.2

In [24]:
archive.size()

3120

In [25]:
best = selBest(best_ind, 1, fit_attr="novelty")[0]
print(best.fitness, best.bd)
best = controller.array_to_dict(best)

(6.205471038818359,) [0.09       0.52       0.45999998 0.5       ]


In [26]:
state = jax.jit(env.reset)(random_key)

In [27]:
jit_step = jax.jit(env.step)
jit_inf = jax.jit(controller.predict)

In [28]:
rollout = []
bd = []
while not state.done:
    rollout.append(state)
    action = jit_inf(best, state.obs)
    state = jit_step(state, action)
    bd.append(state.info["feet_contact"])
    

In [29]:
jnp.mean(jnp.asarray(bd), axis=0)

Array([0.16      , 0.4766667 , 0.46333334, 0.5133333 ], dtype=float32)

In [30]:
state.metrics

{'distance_from_origin': Array(11.143674, dtype=float32),
 'forward_reward': Array(-0.4065323, dtype=float32),
 'reward_contact': Array(-0.00218487, dtype=float32),
 'reward_ctrl': Array(-0.9229876, dtype=float32),
 'reward_forward': Array(-0.4065323, dtype=float32),
 'reward_survive': Array(1., dtype=float32, weak_type=True),
 'x_position': Array(-9.971377, dtype=float32),
 'x_velocity': Array(-0.4065323, dtype=float32),
 'y_position': Array(4.938198, dtype=float32),
 'y_velocity': Array(-0.17809868, dtype=float32)}

In [31]:
HTML(html.render(env.sys, [s.qp for s in rollout]))